In [64]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.cbook import boxplot_stats
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_regression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb

In [69]:
df = pd.read_csv("Final.csv")
df.head()
df.fillna("NaN", inplace=True)
df["ORDER_CREATION_DATE"] = pd.to_datetime(df["ORDER_CREATION_DATE"], format="%Y%m%d")
df["REQUESTED_DELIVERY_DATE"] = pd.to_datetime(df["REQUESTED_DELIVERY_DATE"], format="%Y%m%d")
df.drop(df[(df["ORDER_CREATION_DATE"] > df["REQUESTED_DELIVERY_DATE"])].index, inplace=True)
df["ORDER_AMOUNT"] = df["ORDER_AMOUNT"].str.replace("-", "")
df["ORDER_AMOUNT"] = df["ORDER_AMOUNT"].str.replace(",", ".")
df["RELEASED_CREDIT_VALUE"] = df["RELEASED_CREDIT_VALUE"].str.replace("-", "")
df["RELEASED_CREDIT_VALUE"] = df["RELEASED_CREDIT_VALUE"].str.replace(",", ".")
df["ORDER_CURRENCY"] = df["ORDER_CURRENCY"].replace("HU1", "HUF")
apikey = "347adc49463e4adfafd55bba3192ed32"
url = f"https://openexchangerates.org/api/latest.json?app_id={apikey}"
response = requests.get(url)
data = response.json()
exchange_rates = data["rates"]
def convert_to_usd(row):
    amount = row['ORDER_AMOUNT']
    currency = row['ORDER_CURRENCY']
    if currency != 'USD':
        return float(amount) / exchange_rates.get(currency, 1)
    return amount


df['amount_in_usd'] = df.apply(convert_to_usd, axis=1)
df["UNIQUE_CUST_ID"] = df["CUSTOMER_NUMBER"].astype(str) + df["COMPANY_CODE"].astype(str)
df.sort_values('ORDER_CREATION_DATE',inplace=True)

In [70]:
df['RELEASED_CREDIT_VALUE'] = df['RELEASED_CREDIT_VALUE'].astype(float)
df['amount_in_usd']=df['amount_in_usd'].astype(float)
df['DISTRIBUTION_CHANNEL'] = df['DISTRIBUTION_CHANNEL'].astype(str)
df['DIVISION'] = df['DIVISION'].astype(str)
df['PURCHASE_ORDER_TYPE'] = df['PURCHASE_ORDER_TYPE'].astype(str)
df['CREDIT_CONTROL_AREA'] = df['CREDIT_CONTROL_AREA'].astype(str)
df['CREDIT_STATUS'] = df['CREDIT_STATUS'].astype(str)


In [71]:
df['ORDER_CREATION_DATE'] = pd.to_datetime(df['ORDER_CREATION_DATE'])
df['ORDER_MONTH'] = df['ORDER_CREATION_DATE'].dt.month
df['ORDER_DATE'] = df['ORDER_CREATION_DATE'].dt.day
monthly_data = {}
for month in range(1, 13):
    monthly_data[month] = df[df['ORDER_MONTH'] == month]
train_data = pd.concat([monthly_data[1],monthly_data[2],monthly_data[3],monthly_data[4]], ignore_index=True)
test_data = pd.concat([monthly_data[5],monthly_data[6]], ignore_index=True)

In [73]:
df_adjusted = df[['UNIQUE_CUST_ID','ORDER_CREATION_DATE','amount_in_usd']]

df_adjusted['ORDER_CREATION_DATE'] = pd.to_datetime(df_adjusted['ORDER_CREATION_DATE'])
df_adjusted['ORDER_MONTH'] = df_adjusted['ORDER_CREATION_DATE'].dt.month
monthly_data = {}
for month in range(1, 13):
    monthly_data[month] = df_adjusted[df_adjusted['ORDER_MONTH'] == month]
train_data_adjusted = pd.concat([monthly_data[1],monthly_data[2],monthly_data[3],monthly_data[4]], ignore_index=True)
test_data_adjusted = pd.concat([monthly_data[5],monthly_data[6]], ignore_index=True)

Q1 = train_data_adjusted.amount_in_usd.astype(float).quantile(0.25)
Q3 = train_data_adjusted.amount_in_usd.astype(float).quantile(0.75)
IQR = Q3-Q1
lb = (Q1-2.2*IQR)
ub = (Q3+2.2*IQR)

print(str(lb)+" "+str(ub))
mean_replace = np.mean(train_data_adjusted[~((train_data_adjusted.amount_in_usd.astype(float) < lb) | (train_data_adjusted.amount_in_usd.astype(float) > ub))]['amount_in_usd'].astype(float))
print(str(mean_replace))
train_data_adjusted['amount_in_usd'].loc[(train_data_adjusted['amount_in_usd'] < lb) | (train_data_adjusted['amount_in_usd'] > ub)]=ub

train_data_adjusted = train_data_adjusted.groupby(['UNIQUE_CUST_ID', 'ORDER_CREATION_DATE'], as_index=False).agg({'amount_in_usd': 'sum'})
train_data_adjusted.sort_values('ORDER_CREATION_DATE',inplace=True)
test_data_adjusted = test_data_adjusted.groupby(['UNIQUE_CUST_ID', 'ORDER_CREATION_DATE'], as_index=False).agg({'amount_in_usd': 'sum'})
test_data_adjusted.sort_values('ORDER_CREATION_DATE',inplace=True)

# def difference_in_days(melt, lags, ffday, customer_id_col, create_date_col, net_amount_col):
#     for i in range(ffday, lags+1):
#         melt['Last-'+str(i)+'day_Sales'] = melt.groupby([customer_id_col])[net_amount_col].shift(i)

#     melt = melt.reset_index(drop = True)

#     for i in range(ffday, lags+1):
#         melt['Last-'+str(i)+'day_Diff']  = melt.groupby([customer_id_col])['Last-'+str(i)+'day_Sales'].diff()
#     melt = melt.fillna(0)
#     return melt

# train_data_adjusted = difference_in_days(train_data_adjusted,7,1,'UNIQUE_CUST_ID','ORDER_CREATION_DATE','amount_in_usd')
# test_data_adjusted = difference_in_days(test_data_adjusted,7,1,'UNIQUE_CUST_ID','ORDER_CREATION_DATE','amount_in_usd')

train_data_adjusted.sort_values(['UNIQUE_CUST_ID', 'ORDER_CREATION_DATE'], ascending=True, inplace=True)

for i in range(1, 8):
    train_data_adjusted[f'Last-{i}day_Sales'] = pd.NA
for index, row in train_data_adjusted.iterrows():
    unique_cust_id = row['UNIQUE_CUST_ID']
    order_date = row['ORDER_CREATION_DATE']
    amount = row['amount_in_usd']
    previous_rows = train_data_adjusted.loc[(train_data_adjusted['UNIQUE_CUST_ID'] == unique_cust_id) & (train_data_adjusted['ORDER_CREATION_DATE'] < order_date)]
    for i in range(1, 8):
        if len(previous_rows) >= i:
            train_data_adjusted.at[index, f'Last-{i}day_Sales'] = previous_rows.iloc[-i]['amount_in_usd']

for i in range(1, 8):
    test_data_adjusted[f'Last-{i}day_Sales'] = pd.NA
for index, row in test_data_adjusted.iterrows():
    unique_cust_id = row['UNIQUE_CUST_ID']
    order_date = row['ORDER_CREATION_DATE']
    amount = row['amount_in_usd']
    previous_rows = test_data_adjusted.loc[(test_data_adjusted['UNIQUE_CUST_ID'] == unique_cust_id) & (test_data_adjusted['ORDER_CREATION_DATE'] < order_date)]
    for i in range(1, 8):
        if len(previous_rows) >= i:
            test_data_adjusted.at[index, f'Last-{i}day_Sales'] = previous_rows.iloc[-i]['amount_in_usd']


train_data = train_data.merge(train_data_adjusted, on=['UNIQUE_CUST_ID', 'ORDER_CREATION_DATE'], how='inner')
test_data = test_data.merge(test_data_adjusted, on=['UNIQUE_CUST_ID', 'ORDER_CREATION_DATE'], how='inner')
train_data.rename(columns={'amount_in_usd_x': 'amount_in_usd'}, inplace=True)
train_data.rename(columns={'amount_in_usd_y': 'net_amount_in_usd'}, inplace=True)
test_data.rename(columns={'amount_in_usd_x': 'amount_in_usd'}, inplace=True)
test_data.rename(columns={'amount_in_usd_y': 'net_amount_in_usd'}, inplace=True)

/tmp/ipykernel_2728/3620527201.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjusted['ORDER_CREATION_DATE'] = pd.to_datetime(df_adjusted['ORDER_CREATION_DATE'])
/tmp/ipykernel_2728/3620527201.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjusted['ORDER_MONTH'] = df_adjusted['ORDER_CREATION_DATE'].dt.month
/tmp/ipykernel_2728/3620527201.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

-1543.9661105132504 2291.5963582239947
395.0426580689312


In [74]:
Q1 = train_data.amount_in_usd.astype(float).quantile(0.25)
Q3 = train_data.amount_in_usd.astype(float).quantile(0.75)
IQR = Q3-Q1
lb = (Q1-2.2*IQR)
ub = (Q3+2.2*IQR)

print(str(lb)+" "+str(ub))
mean_replace = np.mean(train_data[~((train_data.amount_in_usd.astype(float) < lb) | (train_data.amount_in_usd.astype(float) > ub))]['amount_in_usd'].astype(float))
print(str(mean_replace))
train_data['amount_in_usd'] = train_data['amount_in_usd'].astype(float)
test_data['amount_in_usd'] = test_data['amount_in_usd'].astype(float)

train_data['amount_in_usd'].loc[(train_data['amount_in_usd'] < lb) | (train_data['amount_in_usd'] > ub)]=ub

-1543.9661105132504 2291.5963582239947
395.0426580689312


/tmp/ipykernel_2728/2223497074.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount_in_usd'].loc[(train_data['amount_in_usd'] < lb) | (train_data['amount_in_usd'] > ub)]=ub


In [75]:
train_data['EXPECTED_DELIVERY_TIME'] = (train_data['REQUESTED_DELIVERY_DATE'] - train_data['ORDER_CREATION_DATE']).dt.days
test_data['EXPECTED_DELIVERY_TIME'] = (test_data['REQUESTED_DELIVERY_DATE'] - test_data['ORDER_CREATION_DATE']).dt.days

categorical_columns = ['SALES_ORG', 'DISTRIBUTION_CHANNEL', 'DIVISION', 'CREDIT_CONTROL_AREA', 'CREDIT_STATUS','UNIQUE_CUST_ID']

le = LabelEncoder()
for column in categorical_columns:
    train_data[column] = le.fit_transform(train_data[column])
    dic = dict(zip(le.classes_, le.transform(le.classes_)))
    test_data[column]=test_data[column].map(dic).fillna(6474)

In [76]:
train_data['RELEASED_CREDIT_VALUE_LOG'] = np.where(train_data['RELEASED_CREDIT_VALUE'] > 0, np.log(train_data['RELEASED_CREDIT_VALUE']), train_data['RELEASED_CREDIT_VALUE'])
train_data['Last-1day_Sales_log'] = np.where(train_data['Last-1day_Sales'] > 0, np.log(train_data['Last-1day_Sales']), train_data['Last-1day_Sales'])
train_data['Last-2day_Sales_log'] = np.where(train_data['Last-2day_Sales'] > 0, np.log(train_data['Last-2day_Sales']), train_data['Last-2day_Sales'])
train_data['Last-3day_Sales_log'] = np.where(train_data['Last-3day_Sales'] > 0, np.log(train_data['Last-3day_Sales']), train_data['Last-3day_Sales'])
train_data['Last-4day_Sales_log'] = np.where(train_data['Last-4day_Sales'] > 0, np.log(train_data['Last-4day_Sales']), train_data['Last-4day_Sales'])
train_data['Last-5day_Sales_log'] = np.where(train_data['Last-5day_Sales'] > 0, np.log(train_data['Last-5day_Sales']), train_data['Last-5day_Sales'])
train_data['Last-6day_Sales_log'] = np.where(train_data['Last-6day_Sales'] > 0, np.log(train_data['Last-6day_Sales']), train_data['Last-6day_Sales'])
train_data['Last-7day_Sales_log'] = np.where(train_data['Last-7day_Sales'] > 0, np.log(train_data['Last-7day_Sales']), train_data['Last-7day_Sales'])


train_data['Last-1day_Sales_log'].fillna(0, inplace=True)
train_data['Last-2day_Sales_log'].fillna(0, inplace=True)
train_data['Last-3day_Sales_log'].fillna(0, inplace=True)
train_data['Last-4day_Sales_log'].fillna(0, inplace=True)
train_data['Last-5day_Sales_log'].fillna(0, inplace=True)
train_data['Last-6day_Sales_log'].fillna(0, inplace=True)
train_data['Last-7day_Sales_log'].fillna(0, inplace=True)

test_data['RELEASED_CREDIT_VALUE_LOG'] = np.where(test_data['RELEASED_CREDIT_VALUE'] > 0, np.log(test_data['RELEASED_CREDIT_VALUE']), test_data['RELEASED_CREDIT_VALUE'])
test_data['Last-1day_Sales_log'] = np.where(test_data['Last-1day_Sales'] > 0, np.log(test_data['Last-1day_Sales']), test_data['Last-1day_Sales'])
test_data['Last-2day_Sales_log'] = np.where(test_data['Last-2day_Sales'] > 0, np.log(test_data['Last-2day_Sales']), test_data['Last-2day_Sales'])
test_data['Last-3day_Sales_log'] = np.where(test_data['Last-3day_Sales'] > 0, np.log(test_data['Last-3day_Sales']), test_data['Last-3day_Sales'])
test_data['Last-4day_Sales_log'] = np.where(test_data['Last-4day_Sales'] > 0, np.log(test_data['Last-4day_Sales']), test_data['Last-4day_Sales'])
test_data['Last-5day_Sales_log'] = np.where(test_data['Last-5day_Sales'] > 0, np.log(test_data['Last-5day_Sales']), test_data['Last-5day_Sales'])
test_data['Last-6day_Sales_log'] = np.where(test_data['Last-6day_Sales'] > 0, np.log(test_data['Last-6day_Sales']), test_data['Last-6day_Sales'])
test_data['Last-7day_Sales_log'] = np.where(test_data['Last-7day_Sales'] > 0, np.log(test_data['Last-7day_Sales']), test_data['Last-7day_Sales'])



test_data['Last-1day_Sales_log'].fillna(0, inplace=True)
test_data['Last-2day_Sales_log'].fillna(0, inplace=True)
test_data['Last-3day_Sales_log'].fillna(0, inplace=True)
test_data['Last-4day_Sales_log'].fillna(0, inplace=True)
test_data['Last-5day_Sales_log'].fillna(0, inplace=True)
test_data['Last-6day_Sales_log'].fillna(0, inplace=True)
test_data['Last-7day_Sales_log'].fillna(0, inplace=True)

train_data['amount_in_usd_log'] = np.where(train_data['amount_in_usd'] > 0, np.log(train_data['amount_in_usd']), train_data['amount_in_usd'])
test_data['amount_in_usd_log'] = np.where(test_data['amount_in_usd'] > 0, np.log(test_data['amount_in_usd']), test_data['amount_in_usd'])

train_data['net_amount_in_usd_log'] = np.where(train_data['net_amount_in_usd'] > 0, np.log(train_data['net_amount_in_usd']), train_data['net_amount_in_usd'])
test_data['net_amount_in_usd_log'] = np.where(test_data['net_amount_in_usd'] > 0, np.log(test_data['net_amount_in_usd']), test_data['net_amount_in_usd'])


/usr/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


TypeError: loop of ufunc does not support argument 0 of type NAType which has no callable log method

In [ ]:

features = ['ORDER_DATE','ORDER_MONTH','RELEASED_CREDIT_VALUE_LOG','net_amount_in_usd_log','EXPECTED_DELIVERY_TIME','SALES_ORG', 'DISTRIBUTION_CHANNEL', 'DIVISION', 'CREDIT_CONTROL_AREA','Last-1day_Sales_log','Last-2day_Sales_log','Last-3day_Sales_log','Last-4day_Sales_log','Last-5day_Sales_log','Last-6day_Sales_log','Last-7day_Sales_log']

target = 'amount_in_usd_log'
X_train = train_data[features]
X_test = test_data[features]
y_train = train_data[target]
y_test = test_data[target]
r2_train_list = []
r2_test_list = []
mse_train_list = []
mse_test_list = []
rmse_train_list = []
rmse_test_list = []

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

r2_train_list.append(r2_train)
r2_test_list.append(r2_test)
mse_train_list.append(mse_train)
mse_test_list.append(mse_test)
rmse_train_list.append(rmse_train)
rmse_test_list.append(rmse_test)

print("Train - R-squared Score:", r2_train)
print("Train - Mean Squared Error:", mse_train)
print("Test - R-squared Score:", r2_test)
print("Test - Mean Squared Error:", mse_test)

Train - R-squared Score: 0.42441501103384105
Train - Mean Squared Error: 4.230591399718808
Test - R-squared Score: 0.27712358552254623
Test - Mean Squared Error: 6.282473200604795


In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 80, #80-100
    'learning_rate': 0.05, #.06 -.07
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 6,#3-5
    'verbose': 1
}
model = lgb.train(params,train_data,num_boost_round=1000)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print("Train - R-squared Score:", r2_train)
print("Train - Mean Squared Error:", mse_train)
print("Test - R-squared Score:", r2_test)
print("Test - Mean Squared Error:", mse_test)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055770 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2970
[LightGBM] [Info] Number of data points in the train set: 855281, number of used features: 14
[LightGBM] [Info] Start training from score 4.583369
Train - R-squared Score: 0.8045358482005713
Train - Mean Squared Error: 1.436675687184422
Test - R-squared Score: 0.3033606708219597
Test - Mean Squared Error: 6.054448351606643
